<a href="https://colab.research.google.com/github/Jose-Gabriel-Rodriguez/CursoIA/blob/main/Practica_3_Detecci%C3%B3n_de_Infecci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image](https://github.com/JoseGabriel-ITD/Probabilidad-y-Estadistica/blob/main/Cintilla2004.png?raw=true)

# **Curso de Inteligencia Artificial Aplicado en Ingeniería Eléctrica**

## Facilitador: ***Dr. José Gabriel Rodríguez Rivas***

### Prática 3: Detección temprana de riesgo de infección postoperatoria

## **Introducción**



In [25]:
# Predicción de infección postoperatoria con Random Forest
# Ejemplo para Ingeniería Biomédica
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

data = pd.read_csv("infecciones.csv")
data.head()

,temp_24h_mean,heart_rate_mean,wbc_count,surgery_duration_min,age,infection
0,37.347700,91.259406,8.451311,98.347873,72,1
1,36.903215,73.807463,13.124629,125.304626,50,1
2,37.453382,81.153449,7.062107,103.599597,62,0
3,38.066121,74.452697,6.602424,111.850347,43,0
4,36.836093,74.786045,7.551769,170.203563,20,0


### **Paso 2. Definir variables predictoras ***(X)*** y variable a predecir (y)**

In [26]:
X = data.drop(columns=["infection"])
y = data["infection"]

### **Paso 3. Dividir dataset en Datos de entrenamiento y prueba**

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                    random_state=42)

### **Paso 4: Entrenamiento del Modelo (Random Forest Regressor)**

In [28]:
# --- Entrenar modelo Random Forest ---
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

### **Paso 5: Predicción y Evaluación**

In [29]:
y_pred = rf_clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred,
                               target_names=["No Infectado", "Infectado"])

print(f"Exactitud del modelo: {acc:.4f}")
print("\nReporte de clasificación:\n", report)
print("\nMatriz de confusión:\n", cm)

Exactitud del modelo: 0.9100

Reporte de clasificación:
               precision    recall  f1-score   support

No Infectado       0.90      0.96      0.93       128
   Infectado       0.92      0.82      0.87        72

    accuracy                           0.91       200
   macro avg       0.91      0.89      0.90       200
weighted avg       0.91      0.91      0.91       200


Matriz de confusión:
 [[123   5]
 [ 13  59]]


### **Paso 6. Interpretación de resultados**

**Exactitud general: 0.91 (91%)**.
* Significa que el modelo clasificó correctamente al 91 % de los pacientes del conjunto de prueba.
* Es un valor sólido considerando que los datos son sintéticos y no se usó ajuste de hiperparámetros ni balanceo especial.

**Matriz de confusión**
|                        | Predicho: No infección | Predicho: Infección |
| ---------------------- | ---------------------- | ------------------- |
| **Real: No infección** | 123 (TN)               | 5 (FP)              |
| **Real: Infección**    | 13 (FN)                | 59 (TP)             |  

* **13 falsos negativos (FN):** pacientes con infección que el modelo clasificó como “No infección”.  
* **5 falsos positivos (FP):** pacientes sanos que el modelo marcó como “Infección”.

*Esto indica que el modelo prioriza la precisión y exactitud global, aunque aún podría mejorarse la sensibilidad (recall) para minimizar falsos negativos (riesgo clínico más relevante).*

**Propuestas de Mejoras al modelo:**
* Integrar variables adicionales: presión arterial, oxigenación, índice de masa corporal, tipo de cirugía.

### **Paso 7. Visualizar Importancia de las variables**

In [30]:
importances = pd.Series(rf_clf.feature_importances_,
                        index=X.columns).sort_values(ascending=False)
print("\nImportancia de variables:")
print(importances)


Importancia de variables:
heart_rate_mean         0.627030
wbc_count               0.209896
temp_24h_mean           0.071466
surgery_duration_min    0.055041
age                     0.036567
dtype: float64


In [31]:
import plotly.express as px
fig_imp = px.bar(
    x=importances.values,
    y=importances.index,
    orientation='h',
    title="Importancia de variables en el modelo Random Forest",
    color=importances.values,
    color_continuous_scale='Tealgrn',
    labels={"x": "Importancia", "y": "Variable"}
)
fig_imp.update_layout(width=600, height=400)
fig_imp.show()


In [33]:
# --- Correlación entre temperatura y riesgo ---
fig_corr = px.scatter(
    data,
    x="temp_24h_mean",
    y="wbc_count",
    color=data["infection"].map({0: "No Infection", 1: "Infection"}),
    title="Relación entre temperatura y conteo de glóbulos blancos",
    labels={"temp_24h_mean": "Temperatura (°C)", "wbc_count":
            "Glóbulos blancos (x10⁹/L)"}
)
fig_corr.update_layout(width=600, height=400)
fig_corr.show()


### **Paso 8. Implementación del modelo para Predicciones sobre nuevos pacientes (ficticios)**

In [32]:
new_patients = pd.DataFrame({
    "temp_24h_mean": [37.2, 38.5, 39.0, 36.8, 38.1, 37.5, 37.0, 38.8],
    "heart_rate_mean": [78, 92, 105, 75, 88, 81, 77, 98],
    "wbc_count": [8, 11, 14, 9, 12, 10, 9, 13],
    "surgery_duration_min": [115, 160, 180, 100, 150, 120, 110, 170],
    "age": [35, 65, 58, 40, 70, 50, 45, 68]
})

probs = rf_clf.predict_proba(new_patients)[:, 1]
preds = (probs > 0.5).astype(int)

new_patients["infection_risk"] = probs
new_patients["prediction"] = np.where(preds == 1, "Infection", "No Infection")

# Clasificación por colores según nivel de riesgo
def risk_level(p):
    if p < 0.5:
        return "Low"
    elif p < 0.85:
        return "Moderate"
    else:
        return "High"

def risk_color(p):
    if p < 0.5:
        return "green"
    elif p < 0.85:
        return "yellow"
    else:
        return "red"

new_patients["Risk Level"] = new_patients["infection_risk"].apply(risk_level)
new_patients["Color"] = new_patients["infection_risk"].apply(risk_color)

print("\n--- Predicciones para nuevos pacientes ---")
print(new_patients[["temp_24h_mean", "heart_rate_mean", "wbc_count", "infection_risk", "prediction", "Risk Level"]])



--- Predicciones para nuevos pacientes ---
   temp_24h_mean  heart_rate_mean  wbc_count  infection_risk    prediction  \
0           37.2               78          8            0.01  No Infection   
1           38.5               92         11            0.94     Infection   
2           39.0              105         14            0.87     Infection   
3           36.8               75          9            0.02  No Infection   
4           38.1               88         12            0.97     Infection   
5           37.5               81         10            0.31  No Infection   
6           37.0               77          9            0.02  No Infection   
7           38.8               98         13            0.90     Infection   

  Risk Level  
0        Low  
1       High  
2       High  
3        Low  
4       High  
5        Low  
6        Low  
7       High  


In [34]:
# --- Dashboard interactivo ---
fig = px.bar(
    new_patients,
    x=new_patients.index,
    y="infection_risk",
    color="Risk Level",
    color_discrete_map={"Low": "green", "Moderate": "yellow", "High": "red"},
    title="Riesgo de infección postoperatoria por paciente",
    labels={"x": "Paciente", "infection_risk": "Probabilidad de infección"}
)
fig.update_layout(width=700, height=400)
fig.show()